In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import calendar
from timer import timer
import datetime as dt

In [2]:
df = pd.read_csv('applications data.csv')

In [3]:
#fix datatype
df['date'] = pd.to_datetime(df['date'],format='%Y%m%d')
df['zip5'] = df['zip5'].apply(lambda x: '{0:0>5}'.format(x))

In [4]:
#fix frivolous values
#ssn
df.loc[df.ssn == 999999999, 'ssn'] = -df[df.ssn == 999999999]['record']
df['ssn'] = df['ssn'].apply(lambda x:'{0:0>9}'.format(x))

#address
df.loc[df.address == '123 MAIN ST', 'address'] = df.loc[
    df.address == '123 MAIN ST', 'record'].apply(lambda x: str(x)+' RECORD')

#dob
df.loc[df.dob == 19070626,'dob'] = -df.loc[df.dob == 19070626]['record']
df['dob'] = df['dob'].apply(lambda x: '{0:0>8}'.format(x))

#homephone
df.loc[df.homephone == 9999999999, 'homephone'] = -df.loc[df.homephone == 9999999999]['record']
df['homephone'] = df['homephone'].apply(lambda x: '{0:0>10}'.format(x))

**Risk table for day of week**

In [5]:
df['dow'] = df.date.apply(lambda x: calendar.day_name[x.weekday()])

In [6]:
train_test = df[df.date < '2016-11-01']

In [7]:
#statistical smoothing
c = 4
nmid = 20
y_avg = train_test['fraud_label'].mean()
y_dow = train_test.groupby('dow')['fraud_label'].mean()
num = train_test.groupby('dow').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num - nmid)/c))
df['dow_risk'] = df.dow.map(y_dow_smooth)

**Risk table for day of month**

In [8]:
df['dom'] = df['date'].dt.day

In [9]:
train_test = df[df.date < '2016-11-01']

In [10]:
#statistical smoothing
c = 10
nmid = 50
y_avg = train_test['fraud_label'].mean()
y_dom = train_test.groupby('dom')['fraud_label'].mean()
num = train_test.groupby('dom').size()
y_dom_smooth = y_avg + (y_dom - y_avg)/(1 + np.exp(-(num - nmid)/c))
df['dom_risk'] = df.dom.map(y_dom_smooth)

**Risk table for zipcode**

In [11]:
c = 4
nmid = 50
y_avg = train_test['fraud_label'].mean()
y_zip = train_test.groupby('zip5')['fraud_label'].mean()
num = train_test.groupby('zip5').size()
y_zip_smooth = y_avg + (y_zip - y_avg)/(1 + np.exp(-(num - nmid)/c))
df['zip_risk'] = df.zip5.map(y_zip_smooth)

In [12]:
df['zip_risk'] = df['zip_risk'].fillna(df['zip_risk'].mean())

In [13]:
df['zip_risk'].isnull().sum()

0

**Create Entities**

In [14]:
df['name'] = df.firstname + df.lastname
df['fulladdress'] = df.address + df.zip5
df['name_dob'] = df.name + df.dob
df['name_fulladdress'] = df.name + df.fulladdress
df['name_homephone'] = df.name + df.homephone
df['fulladdress_dob'] = df.fulladdress + df.dob
df['fulladdress_homephone'] = df.fulladdress + df.homephone
df['dob_homephone'] = df.dob + df.homephone
df['homephone_name_dob'] = df.homephone + df.name + df.dob

In [15]:
for field in list(df.iloc[:,np.r_[3:9, 15:18]].columns):
    df['ssn_' + field] = df.ssn + df[field]

In [16]:
attributes = list(df.iloc[:, np.r_[2,5,7,8,15:33]].columns)

In [17]:
attributes

['ssn',
 'address',
 'dob',
 'homephone',
 'name',
 'fulladdress',
 'name_dob',
 'name_fulladdress',
 'name_homephone',
 'fulladdress_dob',
 'fulladdress_homephone',
 'dob_homephone',
 'homephone_name_dob',
 'ssn_firstname',
 'ssn_lastname',
 'ssn_address',
 'ssn_zip5',
 'ssn_dob',
 'ssn_homephone',
 'ssn_name',
 'ssn_fulladdress',
 'ssn_name_dob']

**Velocity + Day since**

In [18]:
df1 = df.copy()
final = df.copy()
df1['check_date'] = df1.date
df1['check_record'] = df1.record

In [19]:
for entity in attributes:
    df_l = df1[['record','date',entity]]
    df_r = df1[['check_record','check_date',entity]]
    temp = pd.merge(df_l, df_r, left_on = entity, right_on = entity)

    # day since
    day_since_df = temp[temp.record>temp.check_record][['record','date','check_date']].groupby('record')[
        ['date','check_date']].last()
    mapper = (day_since_df.date - day_since_df.check_date).dt.days
    final[entity +'_day_since'] = final.record.map(mapper)
    final[entity +'_day_since'].fillna(365, inplace = True)
    print(f'\n{entity}_day_since ---> Done')

    #velocity
    for offset_t in [0,1,3,7,14,30]:
        count_day_df = temp[(temp.check_date>=(temp.date - dt.timedelta(offset_t))) 
                            & (temp.record>=temp.check_record)]
        col_name = f'{entity}_count_{offset_t}'
        mapper2 = count_day_df.groupby('record')[entity].count()
        final[col_name] = final.record.map(mapper2)
        
        print(f'{entity}_count_{str(offset_t)} ---> Done')


ssn_day_since ---> Done
ssn_count_0 ---> Done
ssn_count_1 ---> Done
ssn_count_3 ---> Done
ssn_count_7 ---> Done
ssn_count_14 ---> Done
ssn_count_30 ---> Done

address_day_since ---> Done
address_count_0 ---> Done
address_count_1 ---> Done
address_count_3 ---> Done
address_count_7 ---> Done
address_count_14 ---> Done
address_count_30 ---> Done

dob_day_since ---> Done
dob_count_0 ---> Done
dob_count_1 ---> Done
dob_count_3 ---> Done
dob_count_7 ---> Done
dob_count_14 ---> Done
dob_count_30 ---> Done

homephone_day_since ---> Done
homephone_count_0 ---> Done
homephone_count_1 ---> Done
homephone_count_3 ---> Done
homephone_count_7 ---> Done
homephone_count_14 ---> Done
homephone_count_30 ---> Done

name_day_since ---> Done
name_count_0 ---> Done
name_count_1 ---> Done
name_count_3 ---> Done
name_count_7 ---> Done
name_count_14 ---> Done
name_count_30 ---> Done

fulladdress_day_since ---> Done
fulladdress_count_0 ---> Done
fulladdress_count_1 ---> Done
fulladdress_count_3 ---> Done
fulla

**Relative Velocity**

In [20]:
for att in attributes:
    for d in ['0','1']:
        for dd in ['3','7','14','30']:
            final[att + '_count_' + d + '_by_' + dd] = final[
                att + '_count_' + d]/(final[att + '_count_' + dd] / float(dd))

**Core Variables**

In [21]:
key_attr = ['ssn','homephone','fulladdress','name_dob','fulladdress_homephone']

for entity1 in key_attr:
    for entity2 in key_attr:
        if entity1 == entity2:
            pass
        else:
            df_l = df1[['record', 'date', entity1, entity2]]
            df_r = df1[['check_record', 'check_date', entity1, entity2]]

            temp = pd.merge(df_l, df_r, left_on = entity1, right_on = entity1)

            temp = temp[(temp.record >= temp.check_record)]
            col_name = f'num_of_{entity2}_for_each_{entity1}'
            mapper = temp.groupby(['record', entity1])[entity2+'_y'].nunique()
            mapper = pd.DataFrame(mapper).reset_index()
            final[col_name] = mapper.loc[(mapper['record'] == final['record']) & (mapper[entity1] == final[entity1])][entity2+'_y']
            print(f'{entity1} with {entity2} is ------> Done')

ssn with homephone is ------> Done
ssn with fulladdress is ------> Done
ssn with name_dob is ------> Done
ssn with fulladdress_homephone is ------> Done
homephone with ssn is ------> Done
homephone with fulladdress is ------> Done
homephone with name_dob is ------> Done
homephone with fulladdress_homephone is ------> Done
fulladdress with ssn is ------> Done
fulladdress with homephone is ------> Done
fulladdress with name_dob is ------> Done
fulladdress with fulladdress_homephone is ------> Done
name_dob with ssn is ------> Done
name_dob with homephone is ------> Done
name_dob with fulladdress is ------> Done
name_dob with fulladdress_homephone is ------> Done
fulladdress_homephone with ssn is ------> Done
fulladdress_homephone with homephone is ------> Done
fulladdress_homephone with fulladdress is ------> Done
fulladdress_homephone with name_dob is ------> Done


In [22]:
final.shape

(1000000, 383)

In [23]:
candidates = final.iloc[:,np.r_[11,13,14,33:383]]

In [24]:
stats_candidates = candidates.describe().T[['min','max','mean','std']]

In [25]:
#stats_candidates.to_csv('candidate_variables_statistics.csv')

In [26]:
final.to_csv('all_candidate_variabels.csv')

In [28]:
candidates.shape

(1000000, 353)